<a href="https://colab.research.google.com/github/LokeshVadlamudi/MachineLearningClass/blob/master/titleVsBody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Lokesh Vadlamudi

```

# Title Vs Body

In [0]:
import pandas as pd
import numpy as np

# Downloading Csv file - Liar Liar dataset

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1cdi4TewC1yXhMiGTzPi3lImhvg2cwSx9'
fluff1, id1 = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('train.tsv')

<built-in function id>


In [0]:
link = 'https://drive.google.com/open?id=1cb_eLjtgf5Oz0152ZAqvgafusLvV_erX'
fluff2, id2 = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('test.tsv')

<built-in function id>


In [0]:
# link = 'https://drive.google.com/open?id=1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9'
# fluff3, id3 = link.split('=')
# print (id) # Verify that you have everything after '='
# downloaded = drive.CreateFile({'id':id3}) 
# downloaded.GetContentFile('valid.tsv')

In [0]:
train_data = '/content/train.tsv'
test_data = '/content/test.tsv'
# valid_data = '/content/valid.tsv'

# Data Preprocessing

In [0]:
cols = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 
            'speakerjobtitle', 'stateinfo','partyaffiliation', 
            'barelytruecounts', 'falsecounts','halftruecounts',
            'mostlytrueocounts','pantsonfirecounts','context']

In [0]:
df_train = pd.read_csv(train_data, sep='\t', names = cols)
df_test = pd.read_csv(test_data, sep='\t', names = cols)
# df_valid = pd.read_csv(valid_data, sep='\t', names = cols)

# Encoding Labels

In [0]:
t_labels = ['mostly-true','half-true','original','true']
def convert_Label(i):
  if i not in t_labels:
    return 0
  else:
    return 1

In [0]:
df_train['labelEncoded'] = df_train.apply(lambda row: convert_Label(row['label']), axis=1)
df_test['labelEncoded'] = df_test.apply(lambda row: convert_Label(row['label']), axis=1)

In [0]:
# df = pd.concat([df_train,df_test])
# df = df_train.append(df_test)

In [0]:
df_train.shape

(10240, 15)

In [0]:
df_mod= df_train[['labelEncoded','headline_text','subject']]

In [0]:
df_mod.head()

,labelEncoded,headline_text,subject
0,0,Says the Annies List political group supports ...,abortion
1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,0,Health care reform legislation is likely to ma...,health-care
4,1,The economic turnaround started at the end of ...,"economy,jobs"


In [0]:
df_mod['headline_text'][0]

'Says the Annies List political group supports third-trimester abortions on demand.'

In [0]:
df_mod['labelEncoded'].value_counts()

1    5752
0    4488
Name: labelEncoded, dtype: int64

In [0]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Removing Punctuations

In [0]:
import string
def remove_punctuations(data):
    d = "".join([i for i in str(data) if i not in string.punctuation])
    return d

In [0]:
remove_punctuations('hello-madam')

'hellomadam'

# Tokenize

In [0]:
from nltk.tokenize import RegexpTokenizer
def tokenize(data):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(data.lower())

In [0]:
tokenize('DEEP LEARNING  model')

['deep', 'learning', 'model']

# Remove Stopwords

In [0]:
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(data):
    data = [i for i in data if i not in stopwords.words('english')]
    return data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
remove_stopwords(['deep', 'is' ,'good'])

['deep', 'good']

# Lemmatization

In [0]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

def lematize(data):
    data = [lemma.lemmatize(i) for i in data]
    return data

In [0]:
lematize(['plays'])

['play']

# Stemming

In [0]:
from nltk.stem.porter import PorterStemmer
stemming = PorterStemmer()
def stem(data):
    data = " ".join([stemming.stem(i) for i in data])
    return data

In [0]:
stem(['playing'])

'play'

In [0]:
df_mod['finalHeaderBody'] = df_mod.apply(lambda row: remove_punctuations(row['headline_text']), axis=1)

df_mod['finalHeaderBody'] = df_mod.apply(lambda row: tokenize(row['finalHeaderBody']), axis=1)

df_mod['finalHeaderBody'] = df_mod.apply(lambda row: remove_stopwords(row['finalHeaderBody']), axis=1)

df_mod['finalHeaderBody'] = df_mod.apply(lambda row: lematize(row['finalHeaderBody']), axis=1)

df_mod['finalHeaderBody'] = df_mod.apply(lambda row: stem(row['finalHeaderBody']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [0]:
# df_mod['finalHeaderBody'] = df_mod.apply(lambda row: stem(row['finalHeaderBody']), axis=1)

In [0]:
df_mod

,labelEncoded,headline_text,subject,finalHeaderBody
0,0,Says the Annies List political group supports ...,abortion,say anni list polit group support thirdtrimest...
1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",declin coal start start natur ga took start be...
2,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,hillari clinton agre john mccain vote give geo...
3,0,Health care reform legislation is likely to ma...,health-care,health care reform legisl like mandat free sex...
4,1,The economic turnaround started at the end of ...,"economy,jobs",econom turnaround start end term
...,...,...,...,...
10235,1,There are a larger number of shark attacks in ...,"animals,elections",larger number shark attack florida case voter ...
10236,1,Democrats have now become the party of the [At...,elections,democrat becom parti atlanta metro area black
10237,1,Says an alternative to Social Security that op...,"retirement,social-security",say altern social secur oper galveston counti ...
10238,0,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",lift u cuban embargo allow travel cuba


# Break the text in subject col

In [0]:
import re
def break_subject(text):
  if type(text) is float:
    return []
  import re
  return re.split('; |, |-',text)

In [0]:
df_mod['finalSubject'] = df_mod.apply(lambda row: break_subject(row['subject']), axis=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_mod.head()

,labelEncoded,headline_text,subject,finalHeaderBody,finalSubject
0,0,Says the Annies List political group supports ...,abortion,say anni list polit group support thirdtrimest...,[abortion]
1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",declin coal start start natur ga took start be...,"[energy,history,job, accomplishments]"
2,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,hillari clinton agre john mccain vote give geo...,"[foreign, policy]"
3,0,Health care reform legislation is likely to ma...,health-care,health care reform legisl like mandat free sex...,"[health, care]"
4,1,The economic turnaround started at the end of ...,"economy,jobs",econom turnaround start end term,"[economy,jobs]"


In [0]:
df_mod['finalSubject'][0]

['abortion']

# Generate Synonyms of the subject words

In [0]:
from nltk.corpus import wordnet 
def generate_Synonyms(wordlist):
    synonyms = set()
    for i in wordlist:
      if type(i) is not str:
        continue
      for syn in wordnet.synsets(i): 
          for l in syn.lemmas(): 
              synonyms.add(l.name())
    return list(synonyms)

In [0]:
df_mod['subjectSynonyms'] = df_mod.apply(lambda row: generate_Synonyms(row['finalSubject']), axis=1)
# df['subjectSynonyms'] = df['finalSubject'].apply(lambda x: generate_Synonyms(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_mod.head()

,labelEncoded,headline_text,subject,finalHeaderBody,finalSubject,subjectSynonyms
0,0,Says the Annies List political group supports ...,abortion,say anni list polit group support thirdtrimest...,[abortion],"[abortion, miscarriage]"
1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",declin coal start start natur ga took start be...,"[energy,history,job, accomplishments]","[achievement, acquisition, accomplishment, att..."
2,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,hillari clinton agre john mccain vote give geo...,"[foreign, policy]","[alien, strange, insurance_policy, policy, for..."
3,0,Health care reform legislation is likely to ma...,health-care,health care reform legisl like mandat free sex...,"[health, care]","[guardianship, forethought, worry, concern, ca..."
4,1,The economic turnaround started at the end of ...,"economy,jobs",econom turnaround start end term,"[economy,jobs]",[]


In [0]:
df_mod = df_mod[['subjectSynonyms','finalHeaderBody','labelEncoded']]

In [0]:
df_mod.head()

,subjectSynonyms,finalHeaderBody,labelEncoded
0,"[abortion, miscarriage]",say anni list polit group support thirdtrimest...,0
1,"[achievement, acquisition, accomplishment, att...",declin coal start start natur ga took start be...,1
2,"[alien, strange, insurance_policy, policy, for...",hillari clinton agre john mccain vote give geo...,1
3,"[guardianship, forethought, worry, concern, ca...",health care reform legisl like mandat free sex...,0
4,[],econom turnaround start end term,1


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Count Vector

In [0]:
def CountVector(data):
    v = CountVectorizer(stop_words='english')
    vectorizer = v.fit_transform(data)
    vectors = vectorizer.toarray()
    return vectors

In [0]:
def combineSynonyms(data):
    return ' '.join(data)

In [0]:
combineSynonyms(['mi','as'])

'mi as'

In [0]:
df_mod['subjectSynonyms'] = df_mod['subjectSynonyms'].apply(lambda x: combineSynonyms(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_mod

,subjectSynonyms,finalHeaderBody,labelEncoded
0,abortion miscarriage,say anni list polit group support thirdtrimest...,0
1,achievement acquisition accomplishment attainm...,declin coal start start natur ga took start be...,1
2,alien strange insurance_policy policy foreign ...,hillari clinton agre john mccain vote give geo...,1
3,guardianship forethought worry concern caution...,health care reform legisl like mandat free sex...,0
4,,econom turnaround start end term,1
...,...,...,...
10235,,larger number shark attack florida case voter ...,1
10236,election,democrat becom parti atlanta metro area black,1
10237,security_measure protection certificate securi...,say altern social secur oper galveston counti ...,1
10238,policy insurance_policy insurance,lift u cuban embargo allow travel cuba,0


# Cosine Similarity

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
def cosine_sim(statement):
  statement=statement.split('@@')
  # print(statement)
  text1,text2=statement[0],statement[1]
  vectorizer = TfidfVectorizer(stop_words='english')
  tfidf = vectorizer.fit_transform([text1, text2])
  sim_score=((tfidf * tfidf.T).A)[0,1]
  # if sim_score*100>0:
  #   return 1
  return sim_score

In [0]:
df_mod['test_string']= df_mod['finalHeaderBody'] +'@@'+df_mod['subjectSynonyms']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_mod

,subjectSynonyms,finalHeaderBody,labelEncoded,test_string
0,abortion miscarriage,say anni list polit group support thirdtrimest...,0,say anni list polit group support thirdtrimest...
1,achievement acquisition accomplishment attainm...,declin coal start start natur ga took start be...,1,declin coal start start natur ga took start be...
2,alien strange insurance_policy policy foreign ...,hillari clinton agre john mccain vote give geo...,1,hillari clinton agre john mccain vote give geo...
3,guardianship forethought worry concern caution...,health care reform legisl like mandat free sex...,0,health care reform legisl like mandat free sex...
4,,econom turnaround start end term,1,econom turnaround start end term@@
...,...,...,...,...
10235,,larger number shark attack florida case voter ...,1,larger number shark attack florida case voter ...
10236,election,democrat becom parti atlanta metro area black,1,democrat becom parti atlanta metro area black@...
10237,security_measure protection certificate securi...,say altern social secur oper galveston counti ...,1,say altern social secur oper galveston counti ...
10238,policy insurance_policy insurance,lift u cuban embargo allow travel cuba,0,lift u cuban embargo allow travel cuba@@policy...


In [0]:
df_mod['cosineSim']=df_mod['test_string'].apply(lambda x: cosine_sim(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_mod

,subjectSynonyms,finalHeaderBody,labelEncoded,test_string,cosineSim
0,abortion miscarriage,say anni list polit group support thirdtrimest...,0,say anni list polit group support thirdtrimest...,0.000000
1,achievement acquisition accomplishment attainm...,declin coal start start natur ga took start be...,1,declin coal start start natur ga took start be...,0.000000
2,alien strange insurance_policy policy foreign ...,hillari clinton agre john mccain vote give geo...,1,hillari clinton agre john mccain vote give geo...,0.000000
3,guardianship forethought worry concern caution...,health care reform legisl like mandat free sex...,0,health care reform legisl like mandat free sex...,0.112172
4,,econom turnaround start end term,1,econom turnaround start end term@@,0.000000
...,...,...,...,...,...
10235,,larger number shark attack florida case voter ...,1,larger number shark attack florida case voter ...,0.000000
10236,election,democrat becom parti atlanta metro area black,1,democrat becom parti atlanta metro area black@...,0.000000
10237,security_measure protection certificate securi...,say altern social secur oper galveston counti ...,1,say altern social secur oper galveston counti ...,0.000000
10238,policy insurance_policy insurance,lift u cuban embargo allow travel cuba,0,lift u cuban embargo allow travel cuba@@policy...,0.000000


In [0]:
# df_mod.predicted_label.value_counts()

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
# accuracy_score(df_mod.predicted_label,df_mod.labelEncoded)

# Downloading Fake news dataset csv

In [0]:
link = 'https://drive.google.com/open?id=1CmI37GcFdB4WCjmANeCYfJe7SuYfqOOX'
fluff1, id1 = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('train_fakeNews.csv')

<built-in function id>


In [0]:
link = 'https://drive.google.com/open?id=18z6S7CzSPgTT7Csj44jeT6M42B3xSek1'
fluff1, id1 = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('test_fakeNews.csv')

<built-in function id>


In [0]:
train_fake = pd.read_csv("/content/train_fakeNews.csv")

In [0]:
test_fake = pd.read_csv("/content/test_fakeNews.csv")

In [0]:
train_fake.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
train_fake.shape

(20800, 5)

In [0]:
train_fake['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [0]:
train_fake = train_fake[['title','text','label']]

In [0]:
train_fake['labelEncoded'] = train_fake['label']

In [0]:
train_fake

,title,text,label,labelEncoded
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0,0
20797,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1,1


In [0]:
train_fake = train_fake.drop(train_fake['label'])

In [0]:
train_fake

,title,text,label,labelEncoded
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,1
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0,0
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...,0,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...,0,0
20797,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...,0,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal...",1,1


In [0]:
train_fake['finalHeaderBody'] = train_fake.apply(lambda row: remove_punctuations(row['text']), axis=1)

# train_fake['finalHeaderBody'] = train_fake.apply(lambda row: tokenize(row['finalHeaderBody']), axis=1)

# train_fake['finalHeaderBody'] = train_fake.apply(lambda row: remove_stopwords(row['finalHeaderBody']), axis=1)

# train_fake['finalHeaderBody'] = train_fake.apply(lambda row: lematize(row['finalHeaderBody']), axis=1)

# train_fake['finalHeaderBody'] = train_fake.apply(lambda row: stem(row['finalHeaderBody']), axis=1)

In [0]:
train_fake['finalHeaderBody'] = train_fake.apply(lambda row: tokenize(row['finalHeaderBody']), axis=1)

In [0]:
# train_fake['finalHeaderBody'] = train_fake.apply(lambda row: remove_stopwords(row['finalHeaderBody']), axis=1)

In [0]:
train_fake['finalHeaderBody'] = train_fake.apply(lambda row: lematize(row['finalHeaderBody']), axis=1)

In [0]:
train_fake['finalHeaderBody'] = train_fake.apply(lambda row: stem(row['finalHeaderBody']), axis=1)

In [0]:
train_fake

In [0]:
train_fake['subjectSynonyms'] = train_fake.apply(lambda row: remove_punctuations(row['title']), axis=1)

train_fake['subjectSynonyms'] = train_fake.apply(lambda row: tokenize(row['subjectSynonyms']), axis=1)

train_fake['subjectSynonyms'] = train_fake.apply(lambda row: remove_stopwords(row['subjectSynonyms']), axis=1)

train_fake['subjectSynonyms'] = train_fake.apply(lambda row: lematize(row['subjectSynonyms']), axis=1)

train_fake['subjectSynonyms'] = train_fake.apply(lambda row: stem(row['subjectSynonyms']), axis=1)

In [0]:
train_fake['test_string']= train_fake['finalHeaderBody'] +'@@'+train_fake['subjectSynonyms']

In [0]:
train_fake['cosineSim']=train_fake['test_string'].apply(lambda x: cosine_sim(x))

In [0]:
train_fake

In [0]:
def sim_label(statement):
  statement=statement.split('@@')
  # print(statement)
  text1,text2=statement[0],statement[1]
  vectorizer = TfidfVectorizer(stop_words='english')
  tfidf = vectorizer.fit_transform([text1, text2])
  sim_score=((tfidf * tfidf.T).A)[0,1]
  if sim_score*100>30:
    return 1
  return 0

In [0]:
train_fake['predicted_label']=train_fake['test_string'].apply(lambda x: sim_label(x))

In [0]:
# train_fake.predicted_label.value_counts()

In [0]:
accuracy_score(train_fake.predicted_label,train_fake.labelEncoded)

#NER

In [0]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

def generate_ner_noun(text):
  doc = nlp(text)
  return [chunk for chunk in doc.noun_chunks]

def generate_ner_verb(text):
  doc = nlp(text)
  return [token for token in doc if token.pos_ == "VERB"]

def spacy_similarity(t1,t2):
  doc1 = nlp(t1)
  doc2 = nlp(t2)
  return doc1.similarity(doc2)

# Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)

In [0]:
generate_ner_noun("hi there how are you , i am fine")

In [0]:
generate_ner_verb("you are running and drinking")

In [0]:
spacy_similarity("hi there how are you , i am fine","you are running and drinking")

In [0]:
train_fake['ner_nouns']=train_fake['finalHeaderBody'].apply(lambda x: generate_ner_noun(x))

In [0]:
train_fake['ner_verbs']=train_fake['finalHeaderBody'].apply(lambda x: generate_ner_verb(x))

In [0]:
train_fake.head()

#sentiment score

In [0]:
pip install vaderSentiment

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
analyzer = SentimentIntensityAnalyzer()
def sentiment_score(text):
    score = analyzer.polarity_scores(text)
    return(score['pos']-score['neg'])

# def sentiment_negative_score(text):
#     score = analyzer.polarity_scores(text)
#     return(score['neg'])

In [0]:
sentiment_score("he is very bad")

In [0]:
train_fake['senti_score_body']=train_fake['finalHeaderBody'].apply(lambda x: sentiment_score(x))

In [0]:
train_fake['senti_score_title']=train_fake['subjectSynonyms'].apply(lambda x: sentiment_score(x))

In [0]:
train_fake.head()

#combining train and test dataset - Liar Liar

In [0]:
df_train = pd.read_csv("/content/drive/My Drive/LiarDataset/train.tsv",sep='\t',header=None)

In [0]:
df_test = pd.read_csv("/content/drive/My Drive/LiarDataset/test.tsv",sep='\t',header=None)

In [0]:
# df = pd.concat([df_train,df_test],ignore_index=True)

In [0]:
df_train=df_train.drop(columns=[0])
df_test=df_test.drop(columns=[0])

columns=[
'label',
'statement',
'subject',
'speaker',
'speaker_job_title',
'state info',
'party affiliation',
'barely_true_counts',
'false_counts',
'half_true_counts',
'mostly_true_counts',
'pants_on_fire_counts',
'context'
]

df_train.columns=columns
df_test.columns=columns

df=pd.concat([df_train,df_test])

In [0]:
df

In [0]:
df['labelEncoded'] = df.apply(lambda row: convert_Label(row['label']), axis=1)

In [0]:
df

In [0]:
df['finalHeaderBody'] = df.apply(lambda row: remove_punctuations(row['statement']), axis=1)
df['finalHeaderBody'] = df.apply(lambda row: tokenize(row['finalHeaderBody']), axis=1)
df['finalHeaderBody'] = df.apply(lambda row: remove_stopwords(row['finalHeaderBody']), axis=1)
df['finalHeaderBody'] = df.apply(lambda row: lematize(row['finalHeaderBody']), axis=1)
df['finalHeaderBody'] = df.apply(lambda row: stem(row['finalHeaderBody']), axis=1)

In [0]:
df

In [0]:
df['finalSubject'] = df.apply(lambda row: break_subject(row['subject']), axis=1)

In [0]:
df['subjectSynonyms'] = df.apply(lambda row: generate_Synonyms(row['finalSubject']), axis=1)

In [0]:
df['subjectSynonyms'] = df['subjectSynonyms'].apply(lambda x: combineSynonyms(x))

In [0]:
df['test_string']= df['finalHeaderBody'] +'@@'+df['subjectSynonyms']

In [0]:
df['simScore']=df['test_string'].apply(lambda x: cosine_sim(x))

In [0]:
def sim_label(statement):
  statement=statement.split('@@')
  # print(statement)
  text1,text2=statement[0],statement[1]
  vectorizer = TfidfVectorizer(stop_words='english')
  tfidf = vectorizer.fit_transform([text1, text2])
  sim_score=((tfidf * tfidf.T).A)[0,1]
  if sim_score*100>0:
    return 1
  return 0

In [0]:
df['sim_label']=df['test_string'].apply(lambda x: sim_label(x))

In [0]:
df.to_csv('lokiLiar.csv')

#function for final colab

In [0]:
def predict_tVsBScore(title,body):
    import string
    from nltk.stem import WordNetLemmatizer
    from nltk.tokenize import RegexpTokenizer
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    from nltk.stem.porter import PorterStemmer
    stemming = PorterStemmer()
    lemma = WordNetLemmatizer()
    def cleanUp(data):
        data = "".join([i for i in str(data) if i not in string.punctuation])
        tokenizer = RegexpTokenizer(r'\w+')
        data = tokenizer.tokenize(data.lower())
        data = [i for i in data if i not in stopwords.words('english')]
        data = [lemma.lemmatize(i) for i in data]
        data = " ".join([stemming.stem(i) for i in data])
        # print(data)
        return data

    import re
    def break_subject(text):
        if type(text) is float:
          return []
        import re
        a = re.split('; |, |-',text)
        # print(a)
        return re.split('; |, |-',text)

    cleanedBody = cleanUp(body)
    subject = break_subject(title)

    from nltk.corpus import wordnet 
    def generate_Synonyms(wordlist):
        synonyms = set()
        for i in wordlist:
          if type(i) is not str:
            continue
          for syn in wordnet.synsets(i): 
              for l in syn.lemmas(): 
                  synonyms.add(l.name())
        # print(list(synonyms))
        return list(synonyms)
    def combineSynonyms(data):
        return ' '.join(data)
    

    finalSubject = generate_Synonyms(subject)
    # print(finalSubject)
    finalSubject1 = combineSynonyms(finalSubject)
    # print(finalSubject1)

    s = cleanedBody + '@@' + finalSubject1
    # print(s)

    from sklearn.feature_extraction.text import TfidfVectorizer
    # create the transform
    def cosine_sim(statement):
        statement=statement.split('@@')
        # print(statement)
        text1,text2=statement[0],statement[1]
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf = vectorizer.fit_transform([text1, text2])
        sim_score=((tfidf * tfidf.T).A)[0,1]
        pred_label = 0
        if sim_score*100>20:
          pred_label = 1
          
        # if sim_score*100>0:
        #   return 1
        return sim_score , pred_label

    import spacy
    
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load('en')
    def spacy_similarity(t1,t2):
        doc1 = nlp(t1)
        doc2 = nlp(t2)
        return doc1.similarity(doc2)

    
    simScore , label = cosine_sim(s)
    simScore2 = spacy_similarity(cleanedBody,finalSubject1)
    
    return label,simScore2

In [0]:
predict_tVsBScore('House','House Dem Aide: We Didn’t Even See Comey’s')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


(0, 0.5582465913340703)

In [0]:
def DATAMINERS_getTitleVsBodyScore(title,body):
    if ( (title == "") | (body == "")):
        return 0
    label, simScore = predict_tVsBScore(title,body)
    return (1 - float(simScore))

In [0]:
DATAMINERS_getTitleVsBodyScore('House','House Dem Aide: We Didn’t Even See Comey’s')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


0.4417534086659297